In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [25]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=2,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 667,396 || all params: 125,316,104 || trainable%: 0.5326


In [26]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [27]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.370100,1.330908,0.740625
200,1.135700,0.577648,0.876563
300,0.429100,0.342862,0.882812
400,0.311000,0.327795,0.892188
500,0.321300,0.322617,0.901563
600,0.309400,0.324917,0.898438
700,0.319900,0.313751,0.901563
800,0.323200,0.306108,0.906250
900,0.308500,0.312165,0.906250
1000,0.291000,0.310514,0.901563


In [29]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-1500'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:58<00:00,  4.28it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-1500.csv


In [28]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3701,1.330908,0.344375,0.740625,0.000048,0.013405,0.039192,1.029448,-0.396250,0.464979
1,200,1.1357,0.577648,0.727679,0.876563,0.000045,0.026810,0.558052,1.966077,-0.148884,0.830150
2,300,0.4291,0.342862,0.870089,0.882812,0.000043,0.040214,0.086238,1.251522,-0.012723,0.985588
3,400,0.3110,0.327795,0.895982,0.892188,0.000040,0.053619,-0.016795,0.948765,0.003795,1.004253
4,500,0.3213,0.322617,0.890625,0.901563,0.000037,0.067024,-0.001317,0.995918,-0.010937,0.987868
5,600,0.3094,0.324917,0.895089,0.898438,0.000035,0.080429,-0.015517,0.952242,-0.003348,0.996273
6,700,0.3199,0.313751,0.893750,0.901563,0.000033,0.093834,0.006149,1.019597,-0.007812,0.991334
7,800,0.3232,0.306108,0.891964,0.906250,0.000030,0.107239,0.017092,1.055835,-0.014286,0.984236
8,900,0.3085,0.312165,0.895536,0.906250,0.000028,0.120643,-0.003665,0.988259,-0.010714,0.988177
9,1000,0.2910,0.310514,0.904018,0.901563,0.000025,0.134048,-0.019514,0.937155,0.002455,1.002723
